This was done in google collabration after getting sevaral errors in AWS sagemaker. I finally ran the whole code. I was able to use google drive as disk. Training was done on 30 epochs.

In [0]:
!wget https://github.com/endgameinc/ember/archive/master.zip
!unzip master.zip
!rm master.zip
!cp -r ember-master/* .
!rm -r ember-master
!pip install -r requirements.txt
!python setup.py install

--2020-04-12 15:03:44--  https://github.com/endgameinc/ember/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/endgameinc/ember/zip/master [following]
--2020-04-12 15:03:45--  https://codeload.github.com/endgameinc/ember/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [      <=>           ]  11.22M  4.95MB/s    in 2.3s    

2020-04-12 15:03:48 (4.95 MB/s) - ‘master.zip’ saved [11769324]

Archive:  master.zip
f9a018632ba108b4e25c33d6c3e2e7a7c4487f58
   creating: ember-master/
  inflating: ember-master/LICENSE.txt  
  inflating: ember-master/README.md  
   creating:

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def make_model():
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  feature_size=2381
  tf.compat.v1.disable_eager_execution()

  keras.backend.clear_session()
  
  #Model architecture
  from tensorflow.keras import layers
  
  model = tf.keras.Sequential()
  model.add(layers.InputLayer(input_shape=(1,feature_size)))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(1500, activation='relu',activity_regularizer=tf.keras.regularizers.l1(l=0.01)))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
  print(model.summary())
  return model

In [0]:
model = make_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1, 2381)           0         
_________________________________________________________________
dense (Dense)                (None, 1, 1500)           3573000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1500)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              1501      
Total params: 3,574,501
Trainable params: 3,574,501
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
!mkdir ember_2017_2
!cp -r drive/My\ Drive/midtermdata/* ember_2017_2/

In [0]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("ember_2017_2/")
metadata_dataframe = ember.read_metadata("ember_2017_2/")

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
labelrows = (y_train != -1)

In [0]:
X_train = X_train[labelrows]

In [0]:
y_train = y_train[labelrows]

In [0]:
len(y_train)

600000

In [0]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()

In [0]:
import h5py
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [0]:
!rm drive/My\ Drive/data/ember_2017_2/X_train.dat
!rm drive/My\ Drive/data/ember_2017_2/y_train.dat
!cp X_train.h5 drive/My\ Drive/data/ember_2017_2/
!cp y_train.h5 drive/My\ Drive/data/ember_2017_2/

rm: cannot remove 'drive/My Drive/data/ember_2017_2/X_train.dat': No such file or directory
rm: cannot remove 'drive/My Drive/data/ember_2017_2/y_train.dat': No such file or directory
cp: cannot create regular file 'drive/My Drive/data/ember_2017_2/': Not a directory
cp: cannot create regular file 'drive/My Drive/data/ember_2017_2/': Not a directory


In [0]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()

In [0]:
for x in range(0,600000,100000):
  mms.partial_fit(X_train[x:x+100000])

In [0]:
with open("drive/My Drive/midtermdata/mms_scaler","wb") as f:
  import pickle
  pickle.dump(mms,f)
  f.close()

In [0]:
import h5py
h5f = h5py.File('X_train.h5','r')
X_train = h5f['X_train'][:]
h5f.close()

In [0]:
X_train = mms.transform(X_train)

In [0]:
import numpy as np
X_train = np.reshape(X_train,(-1,1,2381))
y_train = np.reshape(y_train,(-1,1,1))

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

pkl_base="drive/My Drive/data/pickles/"

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=256,
                epochs=1,
                  validation_data =((X_train[600000-120000:600000],y_train[600000-120000:600000]))
                  )
model_name="my_model_test-tpu3.h5"
model_weights="weights_test-tpu3.h5"
model.save(pkl_base+model_name)
model.save_weights(pkl_base+model_weights)
print(model_name,model_weights," are saved.")

Train on 600000 samples, validate on 120000 samples
600000/600000 [==============================] - 246s 411us/sample - loss: 14.4781 - accuracy: 0.9151 - auc_1: 0.9432 - precision_1: 0.9125 - val_loss: 16.9198 - val_accuracy: 0.9670 - val_auc_1: 0.9708 - val_precision_1: 0.9705
my_model_test-tpu3.h5 weights_test-tpu3.h5  are saved.


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

pkl_base="drive/My Drive/data/pickles/"

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
          loss='binary_crossentropy',
          metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

history = model.fit(X_train, y_train,
                batch_size=128,
                epochs=30,
                  validation_split=.2,
                  callbacks=[callback]
                  )
model_name="my_model_test6.h5"
model_weights="weights_test6.h5"
model.save(pkl_base+model_name)
model.save_weights(pkl_base+model_weights)
print(model_name,model_weights," are saved.")

Train on 480000 samples, validate on 120000 samples
Epoch 1/30
480000/480000 [==============================] - 249s 518us/sample - loss: 10.9244 - accuracy: 0.7073 - auc_3: 0.8197 - precision_3: 0.6944 - val_loss: 13.1204 - val_accuracy: 0.7889 - val_auc_3: 0.9130 - val_precision_3: 0.7248
Epoch 2/30
480000/480000 [==============================] - 249s 518us/sample - loss: 7.7330 - accuracy: 0.7073 - auc_3: 0.8212 - precision_3: 0.7000 - val_loss: 29.0429 - val_accuracy: 0.7804 - val_auc_3: 0.8916 - val_precision_3: 0.7182
Epoch 3/30
480000/480000 [==============================] - 247s 515us/sample - loss: 6.1617 - accuracy: 0.7160 - auc_3: 0.8244 - precision_3: 0.8438 - val_loss: 11.4132 - val_accuracy: 0.8012 - val_auc_3: 0.9190 - val_precision_3: 0.9902
Epoch 4/30
480000/480000 [==============================] - 248s 517us/sample - loss: 4.6572 - accuracy: 0.7040 - auc_3: 0.8123 - precision_3: 0.8465 - val_loss: 12.6347 - val_accuracy: 0.7628 - val_auc_3: 0.9021 - val_precision_3